In [2]:
import numpy as np
import pandas as pd
import os

from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options

In [3]:
data_path = "Wisdom Long List 20210917.xlsx"

In [5]:
df1 = pd.read_excel(data_path, sheet_name="MY Main Long List")
df1.head(30)

,No.,Person in Charge,Company Name,Quick Summary,Company Overview,DNB Company info,Good Candidate Reason,Level of Recommendation\n(✗- ✓✓✓),Type of Company,Unnamed: 9,...,Unnamed: 28,Market Cap \n(MYR Mn),Website Quality\n(✓/✖),Presence,Unnamed: 32,Shareholders,Website,Contact Available?,Contacted?,Met?
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,System Integrator,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ERP only,ERP & Others,...,Net Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,Nas,Abeo International Sdn Bhd,NaN,Abeo is a provider of business consulting and ...,NaN,NaN,NaN,✓,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://abeoinc.com/,NaN,NaN,NaN
3,2.0,Nas,Arvato Systems Malaysia Sdn Bhd,NaN,Arvato is a leading international service prov...,NaN,NaN,NaN,✓,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://abeoinc.com//,NaN,NaN,NaN
4,3.0,Nas,Aspert Innovations Sdn Bhd,NaN,Aspert Innovations Sdn Bhd is a consultancy fi...,NaN,NaN,NaN,✓,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,aspert.com.my,NaN,NaN,NaN
5,4.0,Nas,Aureus Consult Sdn Bhd,NaN,Aureus Consult is an ERP Consulting & services...,NaN,NaN,NaN,✓,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,www.aureus.com.my,NaN,NaN,NaN
6,5.0,Nas,cbs Corporate Business Solutions Malaysia Sdn Bhd,NaN,The company provides their customers with comp...,NaN,NaN,NaN,✓,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cbs-consulting.com/apac/,NaN,NaN,NaN
7,6.0,Nas,Croesus IT Solutions Sdn Bhd,NaN,"Established in 2004, Croesus Group of Companie...",NaN,NaN,NaN,✓,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://croesus.com.my/,NaN,NaN,NaN
8,7.0,Nas,EPIUSE Malaysia Sdn Bhd,NaN,"EPI-USE employs more than 3,000 people in 34 c...",NaN,NaN,NaN,✓,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.epiuse.com/,NaN,NaN,NaN
9,8.0,Nas,E-Outsource Asia Sdn Bhd,NaN,The company's services and solutions include A...,NaN,NaN,NaN,✓,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.e-oasia.com/,NaN,NaN,NaN


In [6]:
df1.columns

Index(['No.', 'Person in Charge', 'Company Name', 'Quick Summary',
       'Company Overview', 'DNB Company info', 'Good Candidate Reason',
       'Level of Recommendation\n(✗- ✓✓✓)', 'Type of Company', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Year Established', 'No. of Employees',
       'Partners/Technology', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Country \n(Source: D&B)', 'DNB URL', 'ZoomInfo URL',
       'Financials (Mn)', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Market Cap \n(MYR Mn)',
       'Website Quality\n(✓/✖)', 'Presence', 'Unnamed: 32', 'Shareholders',
       'Website', 'Contact Available?', 'Contacted?', 'Met?'],
      dtype='object')

In [7]:
df_use = pd.DataFrame(df1.loc[df1["DNB URL"].notna(),"DNB URL"])
df_use["info"] = None
df_use["year"] = None
df_use["country"] = None
df_use["revenue"] = None
df_use

,DNB URL,info,year,country,revenue
2,https://www.dnb.com/business-directory/company...,None,None,None,None
3,https://www.dnb.com/business-directory/company...,None,None,None,None
4,https://www.dnb.com/business-directory/company...,None,None,None,None
6,https://www.dnb.com/business-directory/company...,None,None,None,None
7,https://www.dnb.com/business-directory/company...,None,None,None,None
...,...,...,...,...,...
278,https://www.dnb.com/business-directory/company...,None,None,None,None
279,https://www.dnb.com/business-directory/company...,None,None,None,None
280,https://www.dnb.com/business-directory/company...,None,None,None,None
281,https://www.dnb.com/business-directory/company...,None,None,None,None


In [37]:
import requests
import chromedriver_binary

def func(i,url):
    time.sleep(0.5)
    
    option = Options()
    option.add_argument("--headless")
    driver = webdriver.Chrome()
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source, features="html.parser")
    try:
        year = soup.find("li", {"class":"year_started"})
        year = year.find("span", {"class":"value"}).text
        df_use.loc[i,"year"] = year.split()[0]
        print(year.split()[0])
    except:
        print(i,"year is NAN")
        
    
    info = soup.find("span",{"class":"company_summary"}).text
    print(info)
    
    country = soup.find("span", {"class":"company_country"}).text.split()[0]
    print(country.split()[0])

    
    df_use.loc[i,"info"] = info
    df_use.loc[i,"country"] = country
    
    driver.close()
    
    
    print(i,"finished")

In [36]:
url = df_use.loc[2,"DNB URL"]
func(2,url)

2 year is NAN


AttributeError: 'NoneType' object has no attribute 'text'

In [63]:
for i ,row in df_use[200:].iterrows():
    url = df_use.loc[i,"DNB URL"]
    try:
        func(i,url)
    except:
        print(i)

1997
AIRSTAR (M) SDN. BHD. is located in AMPANG, Selangor, Malaysia and is part of the Building Equipment Contractors Industry. AIRSTAR (M) SDN. BHD. has 90 total employees across all of its locations. (Employees figure is estimated). 
Malaysia
231 finished
2009
AR-RIFQI SDN. BHD. is located in AMPANG, Selangor, Malaysia and is part of the Professional and Commercial Equipment and Supplies Merchant Wholesalers Industry. AR-RIFQI SDN. BHD. has 4 total employees across all of its locations.  
Malaysia
232 finished
233 year is NAN
ASPEK TUMPUAN SDN BHD is located in PETALING JAYA, Selangor, Malaysia and is part of the Computer Systems Design and Related Services Industry. ASPEK TUMPUAN SDN BHD has 14 total employees across all of its locations. (Employees figure is modelled). 
Malaysia
233 finished
234 year is NAN
AUGMENTED TECHNOLOGY SDN. BHD. is located in KAJANG, Selangor, Malaysia and is part of the Computer Systems Design and Related Services Industry. AUGMENTED TECHNOLOGY SDN. BHD. 

2003
SQL TECHNOLOGY SDN. BHD. is located in JOHOR BAHRU, Johor, Malaysia and is part of the Electronics and Appliance Stores Industry. SQL TECHNOLOGY SDN. BHD. has 9 total employees across all of its locations. (Employees figure is modelled). 
Malaysia
265 finished
2008
SURIA BORNEO SDN. BHD. is located in KOTA KINABALU, Sabah, Malaysia and is part of the Other Telecommunications Industry. SURIA BORNEO SDN. BHD. has 25 total employees across all of its locations. (Employees figure is modelled). 
Malaysia
266 finished
269 year is NAN
WEVO TECHNOLOGY SDN. BHD. is located in PETALING JAYA, Selangor, Malaysia and is part of the Management, Scientific, and Technical Consulting Services Industry. WEVO TECHNOLOGY SDN. BHD. has 8 total employees across all of its locations. (Employees figure is estimated). 
Malaysia
269 finished
2004
10 CREATIVE SOLUTIONS SDN. BHD. is located in Cyberjaya, Selangor, Malaysia and is part of the Computer Systems Design and Related Services Industry. 10 CREATIVE 

In [67]:
df_use["country"].value_counts()

Malaysia         187
United States     23
Australia          6
Singapore          5
India              4
Israel             2
Germany            2
Thailand           1
Norway             1
Cyprus             1
Netherlands        1
Switzerland        1
Sweden             1
Hong Kong          1
France             1
Poland             1
Spain              1
Name: country, dtype: int64

In [66]:
df_use.loc[df_use["country"]=="United","country"] = "United States"
df_use.loc[df_use["country"]=="Hong","country"] = "Hong Kong"

In [73]:
import re

for i,row in df_use.iterrows():
    
    tmp = row["info"]
    
    earn_pattern1 = r"([0-9]+)\,([0-9]+)\.([0-9]+) million"
    earn_pattern2 = r"([0-9]+)\.([0-9]+) million"
    earn_pattern3 = r"([0-9]+)\,([0-9]+) in sales"
    earn_pattern4 = r"([0-9]+)\.([0-9]+) billion"
    
    try:
        m1 = re.search(earn_pattern1,tmp)
        m2 = re.search(earn_pattern2,tmp)
        m3 = re.search(earn_pattern3,tmp)
        m4 = re.search(earn_pattern4,tmp)
    except:
        print(tmp)
    
    m = m1 if m1 else m2 if m2 else m3 if m3 else m4
    
    if m:
        if "million" in m[0]:
            out = m[0].split()[0]
            df_use.loc[i,"revenue"] = out
        
        elif "billion" in m[0]:
            out = m[0].split()[0]
            df_use.loc[i,"revenue"] = out
            
        elif "sale" in m[0]:
            out = m[0].split()[0]
            df_use.loc[i,"revenue"] = out
        
        print(out)

104.85
2.48
172.06
23.67
220,467
11.97
511.43
154.30
118.80
5.70
386,160
30.00
1.10
36.60
2.69
1.97
300,000
18.00
118,018
637,457
864,775
833,349
17.35
21.45
4.57
47.50
20.37
5.12
179.73
239.81
40.00
303.21
2.06
None
78.16
3.40
471,498
206.71
None
12.15
None
912.05
14.29
687,310
27.38
None
None
15.46


In [74]:
df_use

,DNB URL,info,year,country,revenue
2,https://www.dnb.com/business-directory/company...,ABEO INTERNATIONAL SDN. BHD. is located in KUA...,2011,Malaysia,None
3,https://www.dnb.com/business-directory/company...,ARVATO SYSTEMS MALAYSIA SDN. BHD. is located i...,2005,Malaysia,None
4,https://www.dnb.com/business-directory/company...,ASPERT INNOVATIONS SDN. BHD. is located in SER...,2011,Malaysia,None
6,https://www.dnb.com/business-directory/company...,cbs Corporate Business Solutions Unternehmensb...,1995,Germany,104.85
7,https://www.dnb.com/business-directory/company...,CROESUS IT SOLUTIONS SDN. BHD. is located in P...,None,Malaysia,None
...,...,...,...,...,...
278,https://www.dnb.com/business-directory/company...,LIBERTY TECHNOLOGY RESOURCES SDN. BHD. is loca...,None,Malaysia,None
279,https://www.dnb.com/business-directory/company...,MUTIARA TEKNOLOGI (M) SDN. BHD. is located in ...,1999,Malaysia,None
280,https://www.dnb.com/business-directory/company...,NEXT SPHERE SDN. BHD. is located in Kuala Lump...,1996,Malaysia,None
281,https://www.dnb.com/business-directory/company...,QUANTUM VIRTUALITY SDN. BHD. is located in KUA...,2000,Malaysia,None


In [76]:
import openpyxl
openpyxl.Workbook()
wb = openpyxl.load_workbook(data_path)
sheet = wb["MY Main Long List"]
sheet

<Worksheet "MY Main Long List">

In [77]:
df_use.to_csv("work3.csv",index = False)